# <center><span style="font-family:Papyrus; font-size:4em;">GENOME</span> </center>
## <center>Algorithms for finding innovative leaders within a class of technology</center>
## <center> $\text{By: Jid}\acute{e}$ $\text{Anene}$  </center>

![Caption for the picture.](https://whoinvented.co/wp-content/uploads/2017/03/Spinning-wheel-invented-295x300.gif)

## Motivation
According to the economic innovation literature investors lack domain knowledge and therefore investors underreact to "innovative knowledge," in the form of patents, patent applications, and patent citations. I posit that investors underreact to "innovative knowledge" regardless of the medium, be it patents, clinical trials, or scientific/technical newspaper articles. So, there could be a bunch scenarios in which extremely positive technical/scientific information is released to the public by a firm, but the market price for the firm lags behind the actual value. Or there could be situations in which there is a divergence between the sentiment in technical/scientific media releases and the news media coverage. To see this, consider the case  for company XXX, this company operates in an industry in which survival relies on being an innovative leader (pharmaceuticals, biotech, gene therapy etc.), and now suppose that the non-scientific new media coverage of company XXX is positive, but the technical/scientific public news releases tells a different story, further suppose that the stock price for company XXX seems to be following the non-scientific news coverage instead of the scientific. If this is the case than there is opportunity. 

For companies that depend on innovation traditional fundamental analysis may not be as important as the analysis of scientific/technical innovative news media releases. In the economic literature this type of information content is called "innovative efficiency," and it is measured as a combination between patents/patent citations scaled by research and development expenditures. There are many papers that find strong evidence between innovative efficiency and future returns, but these papers do not use machine learning techniques, and they focus on causality rather than prediction. This project will focus on prediction. 

Furthermore, the experts who inform the sophisticated investor may be ill-informed since time and time again we have seen that experts underperform the machine: the wine taster can't tell the difference between Jersey wine and 1920s French Riesling, the portfolio manager underperforms the rules based consistent eft/mutual fund, the AI outperforming doctors in diagnosing cancer. So there may be no sophisticated investor at all!

Since technical writing is more structured than regular news media text this type of textual data may be easier to process, and so, it seems promising that ML algorithms can come to some robust predictions.  With this reasoning in hand the primary end goal of this project will be to develop a trading strategy based on the information gleaned from scientific mediums. Specifically, I will focus my efforts on analyzing companies that pursue technologies in genome engineering.

## Very Brief Background on Genome Engineering
Genome engineering is a type of genetic engineering in which DNA is inserted, deleted, modified or replaced in the genome of a living organism. A genome is an organism's complete set of DNA, including all of its genes. Each genome contains all of the information needed to build and maintain that organism. Think of a genome as an organism's complete set of genetic instructions. Unlike early genetic engineering techniques that randomly inserts genetic material into a host genome, genome editing targets the insertions to site specific locations. Basically genome engineering is a method for introducing new genetic elements into organisms. 

At a very abstract level computer programming and genome engineering are the same. 
>Computer programming => Introduce algorithms (*or a set of instructions*) within the constructs of a Turing Machine.<br />Genome engineering => Introduce genetic elements (*or a set of instructions*) within the constructs of Biological Organisms. 


There are 9-Types genome engineering methods  (probably more)
- Cas9/Crispr (RNA)
- TALE (Protein) 
- Group II intron (RNA)
- Meganuclease (Protein) 
- Recombinase (Protien) 
- TtAgo (DNA) 
- $\lambda$-beta/exo MAGE (DNA)
- RecACage (DNA)
- ZnF (Protein) 
    

# Data Sources

First, I collected over 500 patent applications by hand from [AppFT](http://appft.uspto.gov/netahtml/PTO/index.html) that were issued between Jan 2018 - June 2018 and mentioned the genome editing technique "*Crispr*" within the text of the application. Then I put the CPC subgroup id codes located within each patent into the text document `.\data\cpc_data.txt`. 
> CPC stands for *Cooperative Patent Classification*, and the CPC code for a patent classifies the technical features of a patent. Patent classifications make it feasible to search quickly for documents about earlier disclosures similar to or related to the invention for which a patent is applied for, and to track technological trends in patent applications.

A data dictionary for the variables provided by PatentView is located at `.\dictionary\PatenView_DataDict.xlsx`.

I then created the API wrapper `.\src\api_wrapper.py` to query patent data from [PatentView API](http://www.patentsview.org/api/doc.html). There are 3 classes within `.\src\api_wrapper.py`: the class `PatentView` gets data from the [PatentView API](http://www.patentsview.org/api/doc.html), the class `ListToQuery` puts the CPC subgroup id's from `.\data\cpc_data.txt` into a valid PatentView query, and the class `Json_to_DataFrame` converts the Json data returned from [PatentView API](http://www.patentsview.org/api/doc.html) into a Pandas DataFrame. 

The only meaningful text data that you can pull from the [PatentView API](http://www.patentsview.org/api/doc.html) is `patent_abstract`, therefore the [PatentView API](http://www.patentsview.org/api/doc.html) will be used primarily to get numeric features and filter patents based on specific criteria. PatentView.com has bulk data files (available at http://www.patentsview.org/download/) and within this collection of files there are three datasets of interest:
- `.\data\brf_sum_text.tsv.zip`(13.512 GB) : Full text of the patent summary
- `.\data\claim.tsv.zip` (11.457 GB): Full text of the patent claims
- `detail_desc_text` (39.40 GB): Text of detailed description of patent. (*Need permission to access*) 

>The **summary** is supposed to be directed toward the invention, rather than the disclosure as a whole. The summary may point out the advantages of the invention or how it solves problems previously existent in the art (and preferably indicated in the Background of the Invention). In chemical cases the summary should point out in general terms the utility of the invention. If possible, the nature and gist of the invention or the inventive concept should be set forth. 

>The **claims** are the heart of a patent, in that they define the limits of exactly what the patent does, and does not, cover. That is, the patentee has the right to exclude others from making, using or selling, only those things which are described by the claims. 

Since the **claims** seem to be the meat and bones of patents the analysis will focus on parsing `~.\data\claim.tsv.zip` until my request to PatentView for access to `detail_desc_text` is granted.


## Feature & Data Selection

### Data Selection 
`.\src\ExtractData.py` first pulls data from [PatentView API](http://www.patentsview.org/api/doc.html) that match a certain query, and then performs an inner join  <br /> (1-to-1 merge) with `~.\data\claim.tsv.zip` using the **patent_id** variable as the matching key. 

In [1]:
import numpy as np
import os, sys
import pandas as pd
sys.path.append(os.path.join(os.getcwd(),"src"))
from ExtractData import ExtractData
from BagofWords import BagOfWords

#Extract the data
DATA = ExtractData()
DATA.get_data()


The default setting is to pull 10,000 patents/patent applications that fit the following criteria:
- **cpc_subgroup_id** that match at least one subgroup code in `.\data\cpc_data.txt`
- **app_date** between 2007-01-01 and 2018-10-31
- **examiner_group** $\le$1799 and **examiner_group** $\ge$3715
- **cpc_section_id**="G" or **cpc_section_id**="A" or **cpc_section_id**="C" 
- **assignee_type** = 2 or **assignee_type** = 3

> Examiner groups: The examiner group number correspond to *art units*, which is another patent classification type. Basically, the examiner groups $\le$1799 are chemistry and biotech/drug patents, and examiner groups $\ge$3715 corresponds to prosthesis and surgical patents. Please see [Uspto](https://www.uspto.gov/patents-application-process/classes-arranged-art-unit-art-units-1764-2691) for a full description of what this filter covers.

>CPC section ID:  A = Human Necessities, G = Physics, C = Chemistry. A patent can have multiple CPC section listed so this filter is not very effective since a patent gets included if it has at least one section ID that meets the criteria. 

> Assignee type:  2 - US Company or Corporation, 3 - Foreign Company or Corporation

Note that in this context 10,000 is an arbitrary number, and the fact that PatentView gave back exactly 10,000 patents leads to the natural conclusion that the space of patent that satisfy this query is larger than 10,000.  


With these filters in place one would expect that only biotech/chemical/drug companies would be picked up, but from the cross tab below we see that mechanical and electric companies are picked up as well. This is slightly surprising, but with additional context it is expected; when searching the patent applications by hand there were some surprising companies pursuing innovations in genome engineering, Microsoft had submitted patent applications that deal with storing data in DNA, Monsanto had a bunch of applications out for editing crops, and even defense contractor Raytheon had submitted applications concerning the creation of genetic circuits. With these examples it is apparent that applications of genome engineering are not idiosyncratic to biotech companies.


In [2]:
#Get Nber Cross Tab
pd.crosstab(DATA.pdData["nber_category_id"],DATA.pdData["nber_category_title"])

nber_category_title,Chemical,Cmp&Cmm,Drgs&Med,Elec,Mech,Others
nber_category_id,,,,,,
1,3329,0,0,0,0,0
2,0,205,0,0,0,0
3,0,0,2221,0,0,0
4,0,0,0,430,0,0
5,0,0,0,0,2575,0
6,0,0,0,0,0,1240


### Feature Selection

For feature selection I used a generalized *Bag-Of-Words model* called the *n-Gram model*. This model works by selecting unique n-length sequence of words, and hence if $n=1$ the *n-Gram model* is exactly the *Bag-of-Words model* we all know and love. With this generalized model we can pick up common phrases and context instead of just common words. 

I also implement a *part-of-speech tagger* which is a method from the python library `nlkt` that reads text and assigns parts of speech to each word, such as noun, verb, adjective, etc..

In [3]:
from nltk import pos_tag
text = "And now for something completely different".split()
pos_tag(text)

[('And', 'CC'),
 ('now', 'RB'),
 ('for', 'IN'),
 ('something', 'NN'),
 ('completely', 'RB'),
 ('different', 'JJ')]

Here we see that *and* is CC, a coordinating conjunction; *now* and *completely* are RB, or adverbs; *for* is IN, a preposition; *something* is NN, a noun; and *different* is JJ, an adjective (*Example taken from [nlkt documentation](https://www.nltk.org/book/ch05.html)*). 

The file `src/BagofWords.py` contains the Bag-of-Word algorithms. The steps for processing the text data are as follows:
1. Patents have a lot of numbered items so delete all decimal numbers: 123456789
2. Removing all non-alpha numeric characters: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
3. Apply the porter stemmer which reduces words to the root. "punctuation" -> "punctuat"
4. Reduce the case of every word to lower 
5. Delete random white space
6. Tokenize the remaining document

After this either extract data using *n-gram sequences* or apply the *part-of-speech tagger*. When the algorithm is finished we have an encoding of the textual information of patent claims in a sparse matrix.


In [4]:
from BagofWords import BagOfWords

#Get the claims_df
claims_df=DATA.CLAIMS_DF
#Get the corpurs for the claims doc
claims_corpus = list(claims_df["text"].astype(str))

#Get Pos-tagger Bag-of-Words Varient for claims data. 
claims_bow = BagOfWords(pos_method = True, numbers = False)
claims_bow.get_bagofwords(claims_corpus)
claims_bow.set_varient_matrices()
#Tf-idf with sublinear tf scaling
pos_X_tfidf_subl = claims_bow.X_tfidf_subl

# Get 3-Gram data. 
claims_bow = BagOfWords(n = 3, numbers = False)
claims_bow.get_bagofwords(claims_corpus)
claims_bow.set_varient_matrices()
bag3_X_tfidf_subl = claims_bow.X_tfidf_subl

There are a bunch of ways we can adjust/normalize the sparse encoding matrix outputted by the bag-of-words model. 
For this illustration `X_tfidf_subl` (*Term Frequency - Inverse Document Frequency with Sublinear Term-Frequency*) is the only adjusted encoding matrix that we will use since we only have finite number of pages and time, and this can get boring. In `src/BagofWords.py` other methods are available. Please see [Stanford’s NLP Link](https://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) for a description of how Tf-idf weighting works, and [another Stanford Link](https://nlp.stanford.edu/IR-book/html/htmledition/sublinear-tf-scaling-1.html) to see what is up with sublinear tf scaling. (Both links aren't very technical).


### Dimension Reduction & Clustering Algorithms


The dataset outputted by `src/BagofWords.py` is very large and very sparse, and so dimension reductions techniques such as K-means is necessary. For K-means the [*elbow method*](http://www.sthda.com/english/wiki/print.php?id=239) and the [*average silhouette method*](http://www.sthda.com/english/wiki/print.php?id=239) is used to find the optimal cluster, $K^*$.
> K-means clustering works by defining clusters that minimize the within-cluster quadratic cost function. $\min(\sum_{k=1}^k W(C_k))$ where $C_k$ is the $k_{th}$ cluster and $W(C_k)$ is the within-cluster quadratic cost. 

>**Elbow Method for finding $K^*$**
- Compute clustering algorithm (e.g., k-means clustering) for different values of k.
- For each k, calculate the total within-cluster quadratic cost
- Plot the of the total within-cluster quadratic cost
- The location of a bend (knee) in the plot is considered as an indicator of the appropriate number of clusters.

>**Average silhouette method for finding $K^*$**
computes the average silhouette of observations for different values of k. The optimal number of clusters $k$ is the one that maximize the average silhouette over a range of values for $k$.

Below see the graphs for the *elbow method* and the *silhouette method*.


In [ ]:
try: del sys.modules["ClusteringAlgos"]
except: from ClusteringAlgos import ElbowMethod

#Get K-means: Elbow method 20 - 1000, with Part-of-Speach Tagged TFid-Sublinear Text-Feature Matrix
tfid_kmeanModel = ElbowMethod(DATA,end=1000,start = 20, steps = 20 ,note="pos_tfid_subl_20_1000", graph_note= "Pos-Tagged Tfid Sublinear")
tfid_kmeanModel.run(pos_X_tfidf_subl)


#Get K-means: Elbow method 20 - 1000, with 3-Gram TFid-Sublinear Text-Feature Matrix
bag3_kmeanModel = ElbowMethod(DATA,end=1000,start = 20, steps = 20 , note="bag3_tfid_subl_20_1000", graph_note= "3-Ngram Tagged Tfid Sublinear")
bag3_kmeanModel.run(bag3_X_tfidf_subl)



#Note: The above 2 algorithms take like 2-4 hours to run... not sure I always do something else when it is running.



 k-mean perc finish:  4% k-mean perc finish:  6% k-mean perc finish:  8% k-mean perc finish:  10% k-mean perc finish:  12% k-mean perc finish:  14% k-mean perc finish:  16% k-mean perc finish:  18% k-mean perc finish:  20% k-mean perc finish:  22% k-mean perc finish:  24% k-mean perc finish:  26% k-mean perc finish:  28% k-mean perc finish:  30% k-mean perc finish:  32% k-mean perc finish:  34% k-mean perc finish:  36% k-mean perc finish:  38% k-mean perc finish:  40% k-mean perc finish:  42% k-mean perc finish:  44% k-mean perc finish:  46% k-mean perc finish:  48% k-mean perc finish:  50% k-mean perc finish:  52% k-mean perc finish:  54% k-mean perc finish:  56% k-mean perc finish:  58% k-mean perc finish:  60% k-mean perc finish:  62% k-mean perc finish:  64% k-mean perc finish:  66% k-mean perc finish:  68% k-mean perc finish:  70% k-mean perc finish:  72% 

It seems that the elbow graph does not have the kink needed for identification of optimal $K^*$, and so there is no clear answer, therefore the *average silhouette approach* is a more appropriate model for cluster selection.


### Future Direction & Refinements

Ultimately since dimension reduction in this instance is for later processing in a downstream phase of the model, cluster parametrization and cluster model selection highly depends on our predictions downstream. Meaning, once we merge our patent data set with stocks we then can evaluate each method and choose the method that gives the most robust predictions. 

Unfortunately, it’s not a straightforward exercise to match stock data with PatentView data. Patents in PatentView come with the variables `assignee_organization` and `assignee_name` if a company is sponsoring a patent, then the company names appear in these variables. Frequently, patents assigned to a given organization may show alternate spelling or syntax formats;
for example, some patents assigned to International Business Machines Corp. may simply show IBM. For a stupid machine this is very confusing. At [NBER](http://www.nber.org/patents/) there is a cross-walk datafile that allows a merge between PatentView data and a standardized schema for company identifiers. So, fixing the issue of company naming conventions is an almost-trivial exercise, but matching company names to tickers is again a problem. Stock tickers are not unique over time, and the spelling of the company will again be misaligned for the same reasons specified above. So to finish the match one would need to do a fuzzy match, this type of match gives probabilities to two words referring to the same thing, and then match all entries that are above a certain probability threshold. 

**There are a couple things in this project that need to be refined**:
- A more precise and systematic way of extracting the companies we want from PatentView.com. - - Instead of using the hand selected CPC Subgroup codes located in `.\data\cpc_data.txt`, apply a graph search algorithm. The nodes of the graph would be a patent, and the edges of the node could either be the cited patents or all patents that are in the same CPC subgroup technology class.  
- Refining the bag-of-words models.
- Incorporating code to allow nonnegative matrix factorization for dimension reduction, in Jeremy Watt's book it is mentioned that these techniques are commonly applied to NLP problems. 
- General UI changes so that the software is easier to work with. 

**The projects current direction**
1. Use features from patentview.com and the dimensionally reduced text-matrix to predict stock prices, and then form a medium-to-long trading strategy based off of predictions. 
2. Use the TwoSigma data set on kaggle.com that has stock prices + news paper articles and analyze divergence between sentiment in the news vs sentiment in patents. Form a medium-to-short term strategy off of this information,
3. Generalize to [Clinical Trial](https://aact.ctti-clinicaltrials.org/download) data. Repeat step 1 and 2 with clinical trial data.
4. Move to options. 
5. Less finance'y and more research'y: Predict if a clinical trial will receive FDA Approval. Predict if a patent application will have a meaningful impact.

### Future Direction & Refinements

Ultimately since dimension reduction in this instance is for later processing in a downstream phase of the model, cluster parametrization and cluster model selection highly depends on our predictions downstream. Meaning, once we merge our patent data set with stocks we then can evaluate each method and choose the method that gives the most robust predictions. 

Unfortunately, it’s not straight forward exercise to match stock data with patent view data. Patents available from PatentView come with the company identifiers `assignee_organization` and `assignee_name`, and if a company is sponsoring a patent then the company's names will appear in one of these two variables. Frequently, patents assigned to a given organization may show alternate spelling or syntax formats;
for example, some patents assigned to International Business Machines Corp. may simply show IBM. For a stupid machine this is very confusing. At [NBER](http://www.nber.org/patents/) there is a cross-walk datafile that allows a merge between PatentView data and a standardized schema for company identifiers. So, fixing the issue of company naming conventions is an almost-trivial exercise, but matching company names to a dataset containing stock tickers/company names is again a problem. Stock tickers are not unique over time, and the spelling of the company will again be misaligned for the same reasons specified above. So to finish the match a fuzzy match needed: a fuzzy of match gives probabilities to two words referring to the same thing, and then matches all entries that are above a certain probability threshold. 

**There are a couple things in this project that need to be refined**:
- A more precise and systematic way of extracting the companies we want from PatentView.com:  
    - *Algorithmic way*: Instead of using the hand selected CPC subgroup codes located in `.\data\cpc_data.txt`, apply a graph search algorithm. The nodes of the graph would be a patent, and the edges of the node could either be the cited patents or all patents that share at least one CPC subgroup technology code.  
    - *Domain knowledge way*: Find out all the ways that the 9-main genome engineering techniques are mentioned in science (Cas9/Crispr, TALE, Group II intron (RNA), Meganuclease, Recombinase, TtAgo , $\lambda$-beta/exo MAGE, RecACage, ZnF). A very non-trivial task since sifting through scientic information as a computer scientist is dizzying.  
- *Refining the bag-of-words models*: Review NLP literature to find a feature selection algorithm that best fits this project.
- *Incorporating code to allow nonnegative matrix factorization for dimension reduction*: In Jeremy Watt's book it is mentioned that these techniques are commonly applied to NLP problems. 
- General UI changes so that it is easier to change the parameters of the model.

**The projects current direction**
1. Use features from PatentView.com and the dimensionally reduced text-matrix to predict stock prices, and then form a medium-to-long trading strategy based off of these predictions. 
2. Use the TwoSigma data set on kaggle.com that has stock prices + newspaper articles and analyze divergences between sentiment in the news vs sentiment in patents. Form a medium-to-short term strategy off of this information.
3. Generalize to [Clinical Trial](https://aact.ctti-clinicaltrials.org/download) data. Repeat step 1 and 2 with clinical trial data.
    - Less finance'y: Predict if clinical trial will receive FDA Approval. 
4. Move to stock options. 

### Conclusion & Monologue
Companies back in the early 1900s used huge steam engines to power everything. The mechanical power came from a single massive steam engine, which turned a central steel drive shaft that ran along the length of the factory. The whole factory was connected to this big engine, and so steam engines rarely stopped. If a single machine in the factory needed to run, the coal fires needed to be fed. Very stupid, why not use an iPhone? Electricity allowed power to be delivered exactly where and when it was needed, and early manufacturing adopters of electricity to their specific business needs easily dispatched of their steam-engine foes. The companies Ford and Oldsmobile were the early adopters of the electricity wave and its applications to the assembly line. Ransom Olds, patented the assembly line concept, which he put to work in his Olds Motor Vehicle Company factory in the early 1900s. The moving assembly line was developed for the Ford Model T and began operation in 1913. The success of assembly line dominated and spread worldwide, and by 1930 companies that did not adopt this technology risked not being able to compete at all. ([Source: BBC "*Why didn't electricity immediately change manufacturing*"](https://www.bbc.com/news/business-40673694))


 The moral of this story is that investors can underreact to innovative knowledge, but companies can also be slow to react to innovation as well, and the leaders in adopting transformative technology to their business needs often dominate, and the spread of this dominance is swift, brutal, and absolute. This project main purpose is to find early adopters and leaders in innovation, and so this project can be formalized as: '***algorithms to find innovative leaders in classes of technology***.' I've had suggestions to make this tool more general and apply it to other technologies such as Bitcoin. I may do this, but I think the impact of genome engineering will far outpace a novel technique to make a marketplace decentralized such as blockchain. So, I will focus on genome engineering, and then generalize after I finish with the current project.

